In [55]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [56]:
model = ConcreteModel()

#### Reading the data

In [57]:
h_ist_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "h_ist", index_col=[0,1,2])
miu_ic_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "miu_ic", index_col=[0,1])
zeta_is_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "p_is", index_col=[0,1])
p_is_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "p_is", index_col=[0,1])
V_ijs_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "V_ijs", index_col=[0,1,2])
tss_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "T_ss", index_col=[0,1])
tsc_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "T_sc", index_col=[0,1])
R_ijst_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "R_ijst", index_col=[0,1,2,3])
B_ii_s_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "BOM", index_col=[0,1,2])
H_jst_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "H_jst", index_col=[0,1,2])
d_ict_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "d_ict", index_col=[0,1,2])
SS_ist_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "h_ist", index_col=[0,1,2])
Invi0_is_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "Inv_ini", index_col=[0,1])
fc_fjs_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "MRLf", index_col=[0,1,2])
MRL_fjs_data = pd.read_excel(r'/Users/hectorcanas/Documents/Modelo Python Karimi/DATA.xlsx', sheet_name= "MRLf", index_col=[0,1,2])

#### Data for sets

In [58]:
families_names = {'F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11'}

i_of_f = {
    'F1':{1,2,3},
    'F2':{4,5},
    'F3':{6,7},
    'F4':{8,9},
    'F5':{10,11},
    'F6':{12,13},
    'F7':{14,15},
    'F8':{16,17},
    'F9':{18,19},
    'F10':{20,21},
    'F11':{22,23}
         }

all_fam_products = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23}

parent_interm_products = {1,4,6,8,10,12,14,16,18,20,22}

child_finished_products = {24,25,26,27,28,29,30,31,32,33,34}

all_finished_products = {2,3,5,7,9,11,13,15,17,19,21,23,24,25,26,27,28,29,30,31,32,33,34}

facilities = {1,2}
origin_facilities = {1,2}

resources = {1,2}

customers = {1}

time_periods = {1,2,3,4,5,6,7,8,9,10,11,12}

child_parent_qt = {24:{1},
          25:{4},
          26:{6},
          27:{8},
          28:{10},
          29:{12},
          30:{14},
          31:{16},
          32:{18},
          33:{20},
          34:{22}
         }

In [59]:

model.T = Set(initialize=time_periods)
model.C = Set(initialize=customers)
model.S = Set(initialize=facilities)
model.J = Set(initialize=resources)

model.I_ip = Set(initialize=parent_interm_products)
model.I =RangeSet(34)
model.F = Set(initialize=families_names)
model.Products_in_fam = Set(initialize=all_fam_products)

model.FxI = Set(within=model.F*model.Products_in_fam, initialize={(f,i) for f in model.F for i in i_of_f[f]})
model.I_of_f = Param(model.F, within = Any, default= set(), initialize=i_of_f)
model.FI = Set(model.F, initialize=i_of_f)

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    unordered data source (type: set).  This WILL potentially lead to
    nondeterministic behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    beha

In [60]:
"""df = pd.Series({(f,i): 1 for f in model.F for i in i_of_f[f]}).to_frame().reset_index()
df.columns = ['f', 'i', 'adjacency']
pd.crosstab(df['f'],df['i'])
"""

"df = pd.Series({(f,i): 1 for f in model.F for i in i_of_f[f]}).to_frame().reset_index()\ndf.columns = ['f', 'i', 'adjacency']\npd.crosstab(df['f'],df['i'])\n"

#### Variables

In [61]:
model.P_ijst = Var(model.I, model.J, model.S, model.T, within=NonNegativeReals)
model.RL_ijst = Var(model.I, model.J, model.S, model.T, within=NonNegativeReals) 
model.C_ist = Var(model.I, model.S, model.T, within=NonNegativeReals)
model.I_ist = Var(model.I, model.S, model.T, within=NonNegativeReals)
model.S_isct = Var(model.I, model.S, model.C, model.T, within=NonNegativeReals)
model.O_iss_t = Var(model.I, model.S, model.S, model.T, within=NonNegativeReals)
model.I_ict = Var(model.I, model.C, model.T, within= NonNegativeReals)
model.I_dev_ist = Var(model.I, model.S, model.T, within=NonNegativeReals)
model.Y_fjst = Var(model.F, model.J, model.S, model.T, domain=Binary)
model.FRL_fjst = Var(model.F, model.J, model.S, model.T, within=NonNegativeReals)


##ok

#### Parameters

a)  Cost coefficients.

In [62]:
model.h_ist = Param(model.I, model.S, model.T, initialize=h_ist_data['h'].to_dict(), default=0)
model.miu_ic = Param(model.I, model.C, initialize=miu_ic_data['miu_ic'].to_dict(), default=0)
model.zeta_is = Param(model.I, model.S, initialize=zeta_is_data['zeta_is'].to_dict(), default=0)
model.p_is = Param(model.I, model.S, initialize=p_is_data['p_is'].to_dict(), default=0)
model.V_ijs = Param(model.I, model.J, model.S, initialize=V_ijs_data['V_i'].to_dict(), default=0)
model.tss_ = Param(model.S, model.S, initialize=tss_data['T_ss'].to_dict(), default=0)
model.tsc = Param(model.S, model.C, initialize=tsc_data['T_sc'].to_dict(),default=0)
model.fc_fjs = Param(model.F, model.J, model.S, initialize=fc_fjs_data['FCf'].to_dict(), default=0)

##todo ok

b) General data

In [63]:
model.R_ijst = Param(model.I, model.J, model.S, model.T, initialize=R_ijst_data['R_ijst'].to_dict(), default=0)
model.B_ii_s = Param(model.I, model.I, model.S, initialize=B_ii_s_data['B'].to_dict(),default=0)
model.H_jst = Param(model.J, model.S, model.T, initialize=H_jst_data['H'].to_dict(), default=0)
model.d_ict = Param(model.I, model.C, model.T, initialize=d_ict_data['d_ict'].to_dict(), default=0)
model.SS_ist = Param(model.I, model.S, model.T, initialize=SS_ist_data['SS'].to_dict(), default=0)
model.Invi0_is = Param(model.I, model.S, initialize=Invi0_is_data['I0i'].to_dict(), default=0)
model.MRL_fjs = Param(model.F, model.J, model.S, initialize=MRL_fjs_data['MRLf'].to_dict(), default=0)

##todo ok

#### Restrictions  (Case 1: NC > 1)

In [64]:
def prod_amount(model,i,j,s,t):
    return model.P_ijst[i,j,s,t] == model.R_ijst[i,j,s,t]*model.RL_ijst[i,j,s,t]
model.prod_amount = Constraint(model.I,model.J,model.S,model.T, rule=prod_amount)  
##2   ok!

In [65]:
def family_lu_bounds_2(model,f,j,s,t):
        return model.FRL_fjst[f,j,s,t] <= model.H_jst[j,s,t]*model.Y_fjst[f,j,s,t]
model.family_lu_bounds_2 = Constraint(model.F,model.J,model.S, model.T, rule=family_lu_bounds_2)
##3 ok!

In [66]:
def family_lu_bounds_1(model,f,j,s,t):
    return model.FRL_fjst[f,j,s,t] >= model.MRL_fjs[f,j,s]*model.Y_fjst[f,j,s,t]
model.family_lu_bounds_1 = Constraint(model.F, model.J, model.S, model.T, rule=family_lu_bounds_1)
##4 ok!

In [67]:
def product_family(model,f,j,s,t):
    return model.FRL_fjst[f,j,s,t]  == sum(model.RL_ijst[i,j,s,t] for i in model.FI[f])
model.product_family = Constraint(model.F, model.J,model.S,model.T, rule=product_family)
##5  ok!

In [68]:
def family_time(model,j,s,t):
    return sum(model.FRL_fjst[f,j,s,t] for f in model.F) <= model.H_jst[j,s,t]
model.family_time = Constraint(model.J, model.S, model.T, rule=family_time)
##6   ok!! revisar si sería de quitar "i"

In [69]:
def rm_consumption(model,i,s,t):
        return model.C_ist[i,s,t] == sum(model.B_ii_s[fp,i,s] * model.P_ijst[fp,j,s,t] for fp in child_finished_products for j in model.J  )
model.rm_consumption = Constraint(model.I_ip, model.S, model.T, rule=rm_consumption)
##7   ok

In [70]:
def consumption_at_site(model, i,s,t):
        return model.C_ist[i,s,t] == sum(model.O_iss_t[i,sprime,s,t] for sprime in model.S)
model.consumption_at_site = Constraint(model.I_ip, model.S, model.T, rule=consumption_at_site)
##8  ok

In [71]:
"""def material_flow(model,i,s,t):
    if t == 1:
        return model.I_ist[i,s,t] == model.Invi0_is[i,s] + sum(model.P_ijst[i,j,s,t] for j in model.J)- sum(model.O_iss_t[i,s,sprime,t] for sprime in model.S) - sum(model.S_isct[i,s,c,t] for c in model.C)
    return model.I_ist[i,s,t] == model.I_ist[i,s,t-1] + sum(model.P_ijst[i,j,s,t] for j in model.J)- sum(model.O_iss_t[i,s,sprime,t] for sprime in model.S) - sum(model.S_isct[i,s,c,t] for c in model.C)
model.material_flow = Constraint(model.I, model.S, model.T, rule=material_flow)"""

##9 ok

'def material_flow(model,i,s,t):\n    if t == 1:\n        return model.I_ist[i,s,t] == model.Invi0_is[i,s] + sum(model.P_ijst[i,j,s,t] for j in model.J)- sum(model.O_iss_t[i,s,sprime,t] for sprime in model.S) - sum(model.S_isct[i,s,c,t] for c in model.C)\n    return model.I_ist[i,s,t] == model.I_ist[i,s,t-1] + sum(model.P_ijst[i,j,s,t] for j in model.J)- sum(model.O_iss_t[i,s,sprime,t] for sprime in model.S) - sum(model.S_isct[i,s,c,t] for c in model.C)\nmodel.material_flow = Constraint(model.I, model.S, model.T, rule=material_flow)'

In [72]:
"""def shortfalls(model,i,c,t):
    if t == 1:
        return model.I_ict[i,c,t] >= model.d_ict[i,c,t] - sum(model.S_isct[i,s,c,t] for s in model.S)
    return model.I_ict[i,c,t] >= model.I_ict[i,c,t-1] + model.d_ict[i,c,t] - sum(model.S_isct[i,s,c,t] for s in model.S)
model.shortfalls = Constraint(model.I, model.C, model.T, rule=shortfalls)
##10 ok"""

'def shortfalls(model,i,c,t):\n    if t == 1:\n        return model.I_ict[i,c,t] >= model.d_ict[i,c,t] - sum(model.S_isct[i,s,c,t] for s in model.S)\n    return model.I_ict[i,c,t] >= model.I_ict[i,c,t-1] + model.d_ict[i,c,t] - sum(model.S_isct[i,s,c,t] for s in model.S)\nmodel.shortfalls = Constraint(model.I, model.C, model.T, rule=shortfalls)\n##10 ok'

In [73]:
"""def previous_demand_satisfaction(model,i,c,t):
    return sum(model.S_isct[i,s,c,tprime] for s in model.S for tprime in model.T if tprime<=t) <= sum(model.d_ict[i,c,tprime] for tprime in model.T if tprime<=t)
model.previous_demand_satisfaction = Constraint(model.I,model.C,model.T, rule=previous_demand_satisfaction)

##11    ok"""

'def previous_demand_satisfaction(model,i,c,t):\n    return sum(model.S_isct[i,s,c,tprime] for s in model.S for tprime in model.T if tprime<=t) <= sum(model.d_ict[i,c,tprime] for tprime in model.T if tprime<=t)\nmodel.previous_demand_satisfaction = Constraint(model.I,model.C,model.T, rule=previous_demand_satisfaction)\n\n##11    ok'

In [74]:
"""def shortage_to_zero(model,i,s,t):
    return model.I_dev_ist[i,s,t] >= model.SS_ist[i,s,t] - model.I_ist[i,s,t]
model.shortage_to_zero = Constraint(model.I,model.S,model.T, rule=shortage_to_zero)"""

##12   

'def shortage_to_zero(model,i,s,t):\n    return model.I_dev_ist[i,s,t] >= model.SS_ist[i,s,t] - model.I_ist[i,s,t]\nmodel.shortage_to_zero = Constraint(model.I,model.S,model.T, rule=shortage_to_zero)'

In [75]:
"""def upper_bound_4(model,i,c,t):
    return model.I_ict[i,c,t] <= sum(model.d_ict[i,c,tprime] for tprime in model.T if tprime<=t)
model.upper_bound_4 = Constraint(model.I, model.C, model.T, rule=upper_bound_4)"""

'def upper_bound_4(model,i,c,t):\n    return model.I_ict[i,c,t] <= sum(model.d_ict[i,c,tprime] for tprime in model.T if tprime<=t)\nmodel.upper_bound_4 = Constraint(model.I, model.C, model.T, rule=upper_bound_4)'

In [76]:
"""def upper_bound_5(model,i,s,t): ###
    return model.I_dev_ist[i,s,t] <= model.SS_ist[i,s,t]
model.upper_bound_5 = Constraint(model.I, model.S, model.T, rule=upper_bound_5)"""

'def upper_bound_5(model,i,s,t): ###\n    return model.I_dev_ist[i,s,t] <= model.SS_ist[i,s,t]\nmodel.upper_bound_5 = Constraint(model.I, model.S, model.T, rule=upper_bound_5)'

In [77]:
"""def prod_bound_rule(model,i,j,s,t):
    return model.P_ijst[i,j,s,t] <= model.R_ijst[i,j,s,t]*model.H_jst[j,s,t]
model.prod_bound = Constraint(model.I,model.J,model.S,model.T, rule=prod_bound_rule) """  ####upper bound"""

'def prod_bound_rule(model,i,j,s,t):\n    return model.P_ijst[i,j,s,t] <= model.R_ijst[i,j,s,t]*model.H_jst[j,s,t]\nmodel.prod_bound = Constraint(model.I,model.J,model.S,model.T, rule=prod_bound_rule) '

In [78]:
"""def prod_run_lenght_bound_rule(model,i,j,s,t):
    return model.RL_ijst[i,j,s,t] <= model.H_jst[j,s,t]
model.prod_run_lenght_bound = Constraint(model.I,model.J,model.S,model.T, rule=prod_run_lenght_bound_rule)"""  # upper"""

'def prod_run_lenght_bound_rule(model,i,j,s,t):\n    return model.RL_ijst[i,j,s,t] <= model.H_jst[j,s,t]\nmodel.prod_run_lenght_bound = Constraint(model.I,model.J,model.S,model.T, rule=prod_run_lenght_bound_rule)'

In [79]:
"""def supply_demand_upper_rule(model,i,s,c,t):
    return model.S_isct[i,s,c,t] <= sum(model.d_ict[i,c,tprime] for tprime in model.T if tprime<=t)
model.supply_demand_upper = Constraint(model.I,model.S,model.C,model.T, rule=supply_demand_upper_rule) """ # upper"""

'def supply_demand_upper_rule(model,i,s,c,t):\n    return model.S_isct[i,s,c,t] <= sum(model.d_ict[i,c,tprime] for tprime in model.T if tprime<=t)\nmodel.supply_demand_upper = Constraint(model.I,model.S,model.C,model.T, rule=supply_demand_upper_rule) '

#### Objective function 

In [80]:
def obj_min_cost(model):
    return sum(model.V_ijs[i,j,s]*model.P_ijst[i,j,s,t] for i in model.I for j in model.J for s in model.S for t in model.T) \
+ sum(model.p_is[i,s]*model.C_ist[i,s,t] for i in model.I for s in model.S for t in model.T) \
+ sum(model.h_ist[i,s,t]*model.I_ist[i,s,t] for i in model.I for s in model.S for t in model.T) \
+ sum(model.tsc[s,c]*model.S_isct[i,s,c,t] for i in model.I for s in model.S for c in model.C for t in model.T)\
+ sum(model.tss_[s,sprime]*model.O_iss_t[i,s,sprime,t] for i in model.I for s in model.S  for sprime in model.S for t in model.T)\
+ sum(model.zeta_is[i,s]*model.I_ist[i,s,t] for i in model.I for s in model.S for t in model.T) \
+ sum(model.miu_ic[i,c]*model.I_ict[i,c,t] for i in model.I for c in model.C for t in model.T) \
+ sum(model.fc_fjs[f,j,s]*model.Y_fjst[f,j,s,t] for f in model.F for j in model.J for s in model.S for t in model.T)
model.obj_min_cost = Objective(sense=minimize, expr=obj_min_cost)


In [81]:
solver = SolverFactory('gurobi')
results = solver.solve(model, tee=True)
model.display()

Using license file /Users/hectorcanas/gurobi.lic
Academic license - for non-commercial use only - expires 2023-05-29
Read LP format model from file /var/folders/g7/v9zzq2fd06j02mp20pdkd4n40000gn/T/tmpldwp42yk.pyomo.lp
Reading time = 0.01 seconds
x9625: 3793 rows, 5269 columns, 6973 nonzeros
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3793 rows, 5269 columns and 6973 nonzeros
Model fingerprint: 0x73f2b3ac
Variable types: 5005 continuous, 264 integer (264 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+04]
  Objective range  [5e-02, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.